In [ ]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tifffile 
import os


In [ ]:
#read in adatas
b1s3_adata = sc.read_h5ad("/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__527489__AB195-default_adata.h5ad")
b1s4_adata = sc.read_h5ad("/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__b261e5__AB193-default_adata.h5ad")
b2s3_adata = sc.read_h5ad("/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__6b38af__AB202-default_adata.h5ad")
b2s4_adata = sc.read_h5ad("/staging/leuven/stg_00079/projects/analysis/LecMicro/NovaST/00.DeepSeq/MPC__9100be__AB204-default_adata.h5ad")


In [ ]:
# Define a function to subset data 
def assign_species_and_subset(adata, column_name="species", gene_prefix="GRCh38_"):
    adata.var[column_name] = np.where(adata.var['gene_symbol'].str.startswith(gene_prefix), "human", "mouse")
    return adata[:, adata.var[column_name] == "human"]

# Apply  function to all samples
b1s3_adata_human = assign_species_and_subset(b1s3_adata)
b1s4_adata_human = assign_species_and_subset(b1s4_adata)
b2s3_adata_human = assign_species_and_subset(b2s3_adata)
b2s4_adata_human = assign_species_and_subset(b2s4_adata)

# Add spatial coordinates to the `obsm` attribute
for adata_human in [b1s3_adata_human, b1s4_adata_human, b2s3_adata_human, b2s4_adata_human]:
    adata_human.obsm["spatial"] = adata_human.obs[["x", "y"]].values


## Determine rotation needed between images and h5ad coordinates for each sample

### B1S3

In [ ]:
dapi_path = "/staging/leuven/stg_00079/projects/Lec_Micro/100.NovaST/Imaging_DeepSeq/B1S3/TransformedB1S3_DS_c4.tif"
dapi_image = Image.open(dapi_path)


In [ ]:
spatial_coords = adata_human.obsm['spatial']  
x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

In [ ]:
dapi_rotated = dapi_image.rotate(-90, expand=True)  # Rotate 90 degrees to the left


In [ ]:
#Get DAPI image dimensions
img_width, img_height = dapi_rotated.size

# Normalize spatial coordinates to the image dimensions
x_coords_scaled = (x_coords / x_coords.max()) * img_width
y_coords_scaled = (y_coords / y_coords.max()) * img_height

In [ ]:
# Convert the image to a NumPy array
dapi_rotated_array = np.array(dapi_rotated)

plt.figure(figsize=(x_coords_scaled.max()/150,y_coords_scaled.max()/150))

plt.scatter(x_coords_scaled, y_coords_scaled, c='red', s=0.0015, alpha=0.5, label='Spots')
plt.imshow(dapi_rotated_array, cmap='gray')

# Ensure the axes match the image
plt.xlim(0, dapi_rotated_array.shape[1])  
plt.ylim(0, dapi_rotated_array.shape[0]) 


In [ ]:

# Generate the new filename
base_name, ext = os.path.splitext(dapi_path)  # Split filename and extension
new_filename = f"{base_name}_rotated{ext}"  # Append '_rotated' to filename

# Save the rotated image with the same dimensions
tifffile.imwrite(new_filename, dapi_rotated_array)

print(f"Rotated image saved as: {new_filename}")

### B1S4

In [ ]:
dapi_path = "/staging/leuven/stg_00079/projects/Lec_Micro/100.NovaST/Imaging_DeepSeq/B1S4/TransformedB1S4_DS_c4.tif"
dapi_image = Image.open(dapi_path)

In [ ]:
spatial_coords = adata_human.obsm['spatial'] 
x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

In [ ]:
dapi_rotated = dapi_image.rotate(90, expand=True)  # Rotate 90 degrees to the right


In [ ]:
#Get DAPI image dimensions
img_width, img_height = dapi_rotated.size

# Normalize spatial coordinates to the image dimensions
x_coords_scaled = (x_coords / x_coords.max()) * img_width
y_coords_scaled = (y_coords / y_coords.max()) * img_height

In [ ]:
# Convert the image to a NumPy array
dapi_rotated_array = np.array(dapi_rotated)

plt.figure(figsize=(x_coords_scaled.max()/150,y_coords_scaled.max()/150))

plt.scatter(x_coords_scaled, y_coords_scaled, c='red', s=0.0015, alpha=0.5, label='Spots')
plt.imshow(dapi_rotated_array, cmap='gray')

# Ensure the axes match the image
plt.xlim(0, dapi_rotated_array.shape[1]) 
plt.ylim(0, dapi_rotated_array.shape[0])



In [ ]:
base_name, ext = os.path.splitext(dapi_path)  
new_filename = f"{base_name}_rotated{ext}"  

# Save the rotated image with the same dimensions
tifffile.imwrite(new_filename, dapi_rotated_array)

print(f"Rotated image saved as: {new_filename}")

### B2S3

In [ ]:
dapi_path = "/staging/leuven/stg_00079/projects/Lec_Micro/100.NovaST/Imaging_DeepSeq/B2S3/TransformedB2S3_DS_c4.tif"
dapi_image = Image.open(dapi_path)

In [ ]:
spatial_coords = b2s3_adata_human.obsm['spatial'] 
x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

In [ ]:
dapi_rotated = dapi_image.rotate(90, expand=True)  # Rotate 90 degrees to the right


In [ ]:
img_width, img_height = dapi_rotated.size
x_coords_scaled = (x_coords / x_coords.max()) * img_width
y_coords_scaled = (y_coords / y_coords.max()) * img_height

In [ ]:
dapi_rotated_array = np.array(dapi_rotated)

#check rotation matches 
plt.figure(figsize=(x_coords_scaled.max()/150,y_coords_scaled.max()/150))

plt.scatter(x_coords_scaled, y_coords_scaled, c='red', s=0.015, alpha=0.7, label='Spots')
plt.imshow(dapi_rotated_array, cmap='gray')

# Ensure the axes match the image
plt.xlim(0, dapi_rotated_array.shape[1])  
plt.ylim(0, dapi_rotated_array.shape[0]) 


In [ ]:

base_name, ext = os.path.splitext(dapi_path)  # Split filename and extension
new_filename = f"{base_name}_rotated{ext}"  # Append '_rotated' to filename

tifffile.imwrite(new_filename, dapi_rotated_array)

print(f"Rotated image saved as: {new_filename}")

### B2S4

In [ ]:
dapi_path = "/staging/leuven/stg_00079/projects/Lec_Micro/100.NovaST/Imaging_DeepSeq/B2S4/TransformedB2S4_DS_c4.tif"
dapi_image = Image.open(dapi_path)

In [ ]:
spatial_coords = b2s3_adata_human.obsm['spatial'] 
x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

In [ ]:
#Get DAPI image dimensions
img_width, img_height = dapi_rotated.size

# Normalize spatial coordinates to the image dimensions
x_coords_scaled = (x_coords / x_coords.max()) * img_width
y_coords_scaled = (y_coords / y_coords.max()) * img_height

In [ ]:
dapi_rotated = dapi_image.rotate(90, expand=True) #rotate
dapi_rotated_array = np.array(dapi_rotated) #this one needs to be mirrored as well
flipped_image = np.flipud(dapi_rotated_array) 

#check rotation matches 
plt.figure(figsize=(x_coords_scaled.max()/150,y_coords_scaled.max()/150))

plt.scatter(x_coords_scaled, y_coords_scaled, c='red', s=0.015, alpha=0.7, label='Spots')
plt.imshow(flipped_image, cmap='gray')

# Ensure the axes match the image
plt.xlim(0, dapi_rotated_array.shape[1])  # Set x-axis to the width of the image
plt.ylim(0, dapi_rotated_array.shape[0])  # Flip the y-axis to match image coordinates



In [ ]:

# Generate the new filename
base_name, ext = os.path.splitext(dapi_path)  # Split filename and extension
new_filename = f"{base_name}_rotated{ext}"  # Append '_rotated' to filename

# Save the rotated image with the same dimensions
tifffile.imwrite(new_filename, flipped_image)

print(f"Rotated image saved as: {new_filename}")